In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
cd /content/drive/My Drive/minor_project

/content/drive/My Drive/minor_project


In [4]:
import numpy as np
import pandas as pd
from PIL import Image
import os
import scipy.ndimage

In [5]:
IMG_SIZE=(1024,1024)
print(IMG_SIZE[0])
print(IMG_SIZE[1])

1024
1024


In [6]:
import cv2
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import os

DATADIR = "/content/drive/My Drive/minor_project"

CATEGORIES = ["Issue_boards", "Ok_boards"]




In [ ]:
data = []


def create_data():
    for category in CATEGORIES:  

        path = os.path.join(DATADIR,category) 
        class_num = CATEGORIES.index(category)  
        for img in tqdm(os.listdir(path)):  
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_COLOR)  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE[0], IMG_SIZE[1]))  # resize to normalize data size
                data.append(np.array([new_array, class_num])) # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass
            #except OSError as e:
            #    print("OSErrroBad img most likely", e, os.path.join(path,img))
            #except Exception as e:
            #    print("general exception", e, os.path.join(path,img))
 
create_data()


  4%|▍         | 3/68 [00:00<00:20,  3.11it/s]

In [ ]:
print(len(data))

In [ ]:
import random

random.shuffle(data)

In [ ]:
for sample in data[:10]:
    print(sample[1])
    

In [ ]:
X = []
y = []

for features,label in data:
    X.append(features)
    y.append(label)

print(X[1].reshape(-1,IMG_SIZE[0], IMG_SIZE[1], 3))

X = np.array(X).reshape(-1,IMG_SIZE[0], IMG_SIZE[1], 3)
X=X/255;
X.size
y=np.array(y)

In [ ]:
X.shape

In [ ]:
len(X)

In [ ]:
!pip install tensorflow-GPU

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D


In [ ]:
plt.imshow(X[220])

In [ ]:
model = Sequential()

model.add(Conv2D(256, (5, 5), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

In [ ]:

model.add(Dense(16))


In [ ]:

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])



In [ ]:
history=model.fit(X, y, batch_size=2, epochs=10, validation_split=0.3)


In [ ]:
model.summary()

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

# Retrieve a list of list results on training and validation data
# sets for each training epoch
loss = history.history['loss']
val_loss = history.history['val_loss']

# Get number of epochs
epochs = range(len(acc))

# Plot training and validation accuracy per epoch
plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title('Training and validation accuracy')

plt.figure()

# Plot training and validation loss per epoch
plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.title('Training and validation loss')